In [0]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
#!wget http://magnitude.plasticity.ai/word2vec/heavy/GoogleNews-vectors-negative300.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

In [0]:
def construct_sentence_vector(command):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  cnt = 0
  for word in command.split():
    word_vector = vectors.query(word)
    # TODO - Do something
    sentence_vector = sentence_vector + word_vector
    cnt+=1
  sentence_vector = sentence_vector/cnt
  return sentence_vector

def find_most_similar_command(user_command, known_commands):
  # TODO - Do something
  user_sent = construct_sentence_vector(user_command)
  known_sent = []
  for command in known_commands:
    known_sent.append(construct_sentence_vector(command))
  #ans = vectors.similarity(user_sent, known_sent) 
  known_sent = np.array(known_sent)
  sim = vectors.similarity(user_sent, known_sent).tolist()
  # ind = sim.index(max(sim))
  # print(sim)
  # return known_commands[ind]
  return sim


##Sentence Embeddings

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from random import randint

import numpy as np
import torch
import string

In [87]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [20]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  28.0M      0  0:00:05  0:00:05 --:--:-- 31.4M


In [22]:
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [0]:
model.set_w2v_path('drive/Shared drives/CIS 700 Project/GloVe/glove.840B.300d.txt')

In [30]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [38]:
embeddings = model.encode(known_commands, bsize=128, tokenize=False, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 77/78 (98.7%)
Speed : 22.6 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 9


In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [0]:
def find_similar_command(user_command, known_commands):

  sim1 = find_most_similar_command(user_command, known_commands)
  sim = []
  for k in known_commands:
    sim.append(cosine(model.encode(user_command)[0], model.encode(k)[0]))
  # sim = [float(i)/sum(sim) for i in sim]
  for i in range(len(sim)):
    sim[i] += sim1[i]
  idx = sim.index(max(sim))
  print(sim)
  print(idx)
  return known_commands[idx]


In [0]:
known_commands = ['Wash each orange well', 'Slice the tops off each orange', 'Scoop the flesh out of each orange', 'Draw a basic face design on the outside of one orange', 'Cut out the design with nail scissors', 'Repeat for the other oranges', "Place a votive candle or tea light candle inside the orange Jack O'Lanterns", 'Arrange in a display', 'Light the candles']

In [69]:
known_commands

['Wash each orange well',
 'Slice the tops off each orange',
 'Scoop the flesh out of each orange',
 'Draw a basic face design on the outside of one orange',
 'Cut out the design with nail scissors',
 'Repeat for the other oranges',
 "Place a votive candle or tea light candle inside the orange Jack O'Lanterns",
 'Arrange in a display',
 'Light the candles']

In [74]:
find_similar_command("draw a design", known_commands)

[0.8641574704910675, 0.9469324270294681, 0.9366446084109368, 1.0578582026062002, 0.9954039667133232, 0.9267725999960715, 0.9226085507618513, 0.9366271201618783, 0.9351094851343895]
3


'Draw a basic face design on the outside of one orange'

##Description

In [0]:
def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects

def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
                if len(w) != 0:
                    tokens.append(''.join(w))
                    w = []
                tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens


In [0]:
def get_description(title):
  DATA = ""
  tit=""
  if title.split()[0].lower()=="how" and title.split()[1].lower()=="to":
    tit += "You want "
    tit += " ".join(title.split()[1:])
  # else:

  DATA += (tit + '\n')
  

  objects = identifyObjects(title)
  # print(objects)
  if len(objects)==1:
    s = "You have a " + objects[0] + "\n"
    DATA += s
  elif len(objects)>1:
    ob += ', '.join(objects)
    s = "You have " + ob + '\n'
    DATA += s
  
  print(DATA)


In [95]:
get_description("How to Make a Jack O'Lantern from an Orange")

['Orange']
You want to Make a Jack O'Lantern from an Orange
You have a Orange



In [94]:
nltk.pos_tag(tokenize("How to Make a Jack O'Lantern from an Orange"))

[('How', 'WRB'),
 ('to', 'TO'),
 ('Make', 'VB'),
 ('a', 'DT'),
 ('Jack', 'NNP'),
 ('O', 'NNP'),
 ("'", 'POS'),
 ('Lantern', 'NNP'),
 ('from', 'IN'),
 ('an', 'DT'),
 ('Orange', 'NN')]